# Introduction 

In this lab we are going to create a basic Artificial Neural Network (ANN) from scratch. The following steps are the general steps to create any ANN. The problem we're goning to solve is a classification problem which is close to the Computer Vission model that wee're going to build next.  

# Step 1: Install the libraries

| Library | Description | 
| --- |---| 
| Tensorflow | Is is an opensource library for creating Deep-Learning models |
| Keras | Keras runs on top of Theano and Tensorflow which wraps the operations of the underlying libraris and keeps the code short | 
| Theano | It is an Opensource fast numerical computation library based on Numpy syntax that runs on the GPU|  
| Pandas | for tabular data processing | 
| Scikit-learn | for Basic ML operations | 

__INstallation command__

```
python3 -m pip install theano tensorflow keras
```

In [4]:
# verify installtion 
import keras
import tensorflow
#import theano

# Step 2: Load Dataset

The classifier that we're going to build is as follows. 
A fictional customer dataset of a bank is given of arounf 10000 customers. The bank has seen an unusual churn rate (the rate at which customer leaves their service provider). The bank wants to analyse their sample datasets to classify based on the customer attributes, which customer has likelyhood of leaving the bank. 

The customer attributes are as follows. 

| Attribute | Description |
| --- | ---| 
| Row Number | Index number at the file | 
| Customer ID | Unique ID to a customer |
| Surname | surname of the customer |
| Credit score |  Credit score of the customer |
| Geography | Which branch the customer is located | 
| Gender | Gender of the customer | 
| Age | Age of the Customer | 
| Tenure | How long the the customer is with the Bank | 
| Number of Product | How many services from the bank the customer has opted for | 
| Has CrCard | Does the customoer has a credit card | 
| Is active Member | is the customer an active memeber of the bank | 
| Estimated Salary | Estimated salary of the customer |
| Exited | has the customer left the bank | 

In [5]:
import pandas as pd    # import pandas to read CSV file 
import sklearn as sk   # data-pre-processing 

In [6]:
dataset_file = 'Data/dataset.csv'      # locating the dataset file
dataset = pd.read_csv(dataset_file)    # loading dataset
dataset  #print first and last 5 rows 

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


# Step 3: Data Pre-processing 

Data pre-processing is the preceeding step before building an ANN. It proceses the data in way such the the ANN does not face the problem of the __Curse of Dimentionality__.  The following steps involves the data pre-processing. 

## 3.1. Dependent & Independent variables segrigation 
In this example, clearly the __Exited__ attribute dependent variable but the independent variable needs to be selected based on their nature. The first step is to manually drop the attributes which does not contribute to the prediction. 

__Notation__ <br>
independent variables = $X=\{x_i | \forall i \in [0,n]\}$ and Dependent Variable = $y$ such that, $y=h(X)$ where the ANN will create the deep learning model in the form of a hypothessis $h$. 

Now, from remaining variables, we'll filter out those which intutively doesn't contribute to the prediction. 


|Index| atrriute | Included as Independent Variable ? |
|--| -- |--| 
|0| __RowNumber__ | No | 
|1| __CustomerID__ | No | 
|2| __Surname__ | No | 
|3| __CreditScore__ | Yes | 
|4| __Geography__ | Yes |
|5| __Gender__ | Maybe | 
|6| __Age__ | Yes |
|7| __Tanure__ | Yes|
|8| __Balance__ | Yes|
|9| __NumberOfProducts__ |Yes|
|10|__HasCrCard__| Yes|
|11|__isActiveMemeber__ |Yes|
|12|__EstimatedSalary__|Yes|

At this stage, we know which attributes could have influence in the outcome but what we don't know is the amount of influence they have. Which the ML model will find. 

In [7]:
dataset.columns # get all colums 

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [8]:
# list all independent and dependent attributes
independet_attr = ['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 
                   'NumOfProducts', 'HasCrCard','IsActiveMember', 'EstimatedSalary']

dependent_attr = ['Exited']
# Filter from the dataset and coonvert into numpy array
X = dataset[independet_attr].iloc[:,:].values 
y = dataset[dependent_attr].iloc[:,:].values

Have a look on the $X$ matrix and $y$ vector.

In [9]:
print(f'X=\n{X} \n') 
print(f'y=\n{y}')

X=
[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]] 

y=
[[1]
 [0]
 [1]
 ...
 [1]
 [1]
 [0]]


In [10]:
X.shape

(10000, 10)

## 3.2. One-hot encoding 

In [11]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder  #Import classes
from sklearn.compose import ColumnTransformer
import numpy as np

In [12]:
col_transformer = ColumnTransformer([('geography', OneHotEncoder(), [1]),
                                     ('gender',OneHotEncoder(),[2])], 
                                     remainder='passthrough'
                            )

X = col_transformer.fit_transform(X)

In [13]:
print(X)
print(X.shape)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]
(10000, 13)


In [14]:
# remove dummy variable trap
X=np.delete(X,0,axis=1)  # for the first OHE
X=np.delete(X,3,axis=1)  # for the second OHE
print(X)
print(X.shape)

[[0.0 0.0 1.0 ... 1 1 101348.88]
 [0.0 1.0 1.0 ... 0 1 112542.58]
 [0.0 0.0 1.0 ... 1 0 113931.57]
 ...
 [0.0 0.0 1.0 ... 0 1 42085.58]
 [1.0 0.0 0.0 ... 1 0 92888.52]
 [0.0 0.0 1.0 ... 1 0 38190.78]]
(10000, 11)


## 3.3. Train-Test Split

In [15]:
from sklearn.model_selection import train_test_split
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=test_size, random_state=0)

In [16]:
print(f'size of X_train = {X_train.shape}, X_shape = {X_test.shape}')
print(f'size of Y_train = {y_train.shape}, Y_shape = {y_test.shape}')

size of X_train = (8000, 11), X_shape = (2000, 11)
size of Y_train = (8000, 1), Y_shape = (2000, 1)


In [17]:
## 3.4. Feature Scaling 

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [18]:
print(X_train)

[[-0.5698444   1.74309049  1.09168714 ...  0.64259497 -1.03227043
   1.10643166]
 [ 1.75486502 -0.57369368 -0.91601335 ...  0.64259497  0.9687384
  -0.74866447]
 [-0.5698444  -0.57369368  1.09168714 ...  0.64259497 -1.03227043
   1.48533467]
 ...
 [-0.5698444  -0.57369368 -0.91601335 ...  0.64259497 -1.03227043
   1.41231994]
 [-0.5698444   1.74309049  1.09168714 ...  0.64259497  0.9687384
   0.84432121]
 [ 1.75486502 -0.57369368  1.09168714 ...  0.64259497 -1.03227043
   0.32472465]]


# Step 4: Build the ANN

Building the ANN is the next task. We're going to use the Keras library for building an ANN. There are two ways an ANN can be defined. First, using a Graph the describes the topology of the ANN. Second, using a Sequential Model that adds layes sequentially. In our example we're going to use the Sequential model as the makes the code more scalable and flexible compared to Graph based model. 

In [19]:
import keras    # import Keras using TF backend 
from keras.models import Sequential # To initialize the Neural Network
from keras.layers import Dense      # To build the neural network

In [20]:
# 4.1. initialize the ANN
classifier = Sequential()    # build an object of the Sequential class called classifier 

In [21]:
# 4.2. add layes
classifier.add(Dense(6, input_dim =11, activation='relu'))   #input 
classifier.add(Dense(6, activation='relu' ))   #hidden 
classifier.add(Dense(1, activation='sigmoid' )) # output

# Step 5 Compile the ANN

In [22]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 6 : Fit ANN to the training Set 

In [23]:
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4854 - accuracy: 0.7899
Epoch 2/100
800/800 [==============================] - 1s 2ms/step - loss: 0.4339 - accuracy: 0.8041
Epoch 3/100
800/800 [==============================] - 1s 2ms/step - loss: 0.4240 - accuracy: 0.8077
Epoch 4/100
800/800 [==============================] - 1s 2ms/step - loss: 0.4188 - accuracy: 0.8127
Epoch 5/100
800/800 [==============================] - 1s 2ms/step - loss: 0.4141 - accuracy: 0.8173
Epoch 6/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4105 - accuracy: 0.8194
Epoch 7/100
800/800 [==============================] - 1s 2ms/step - loss: 0.4071 - accuracy: 0.8254
Epoch 8/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4044 - accuracy: 0.8267
Epoch 9/100
800/800 [==============================] - 1s 2ms/step - loss: 0.4019 - accuracy: 0.8286
Epoch 10/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3987 - accura

# Step 7: Get Confution Matrix 

In [24]:
y_pred = classifier.predict(X_test)    # returns probability [0,1]
y_pred = (y_pred > 0.5)   # returns binary classification {0,1}

In [26]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, y_pred)

In [34]:
print(conf_mat)
print('=========================================')
print(f'Correct Predictions: {conf_mat[0][0]}')
print(f'False Negatives : {conf_mat[0][1]}')
print(f'False Positivs : {conf_mat[1][0]}')
print(f'Wrong Predictions : {conf_mat[1][1]}')
print('-------------------------------------')
print(f'Accuracy = {round(conf_mat[0][0] / (conf_mat[0][0] + conf_mat[0][1] + conf_mat[1][0] + conf_mat[1][1]),2)*100}%')

[[1519   76]
 [ 200  205]]
Correct Predictions: 1519
False Negatives : 76
False Positivs : 200
Wrong Predictions : 205
-------------------------------------
Accuracy = 76.0%


Congratualtions !! yourve built your first Artificial Neural Network from scratch :) 